<img src="https://lund-observatory-teaching.github.io/lundpython/imgs/front_2.png" width="1400">

<h1><center> Course website </center></h1>

To download all lecture files and see the schedule, please visit:

[lund-observatory-teaching.github.io/lundpython/](https://lund-observatory-teaching.github.io/lundpython/)

Each lecture contains (as notebooks)
- Manual 
- Exercises
- Presentation

---

### Flexibility vs efficieny

Python is very good at handling sequences of heterogenous data (different data types).

In [ ]:
for number in [1, '2', [3]]:
    print(f'2*{number} = {2*number}')

Python achieves this flexibility by checking what multiplying the variables means every single time it encounters a multiplication.

But these checks cause computational overhead, even if the data is homogenous!

<img src="https://lund-observatory-teaching.github.io/lundpython/imgs/numpy.png" width="300"/>

### What is NumPy?

From [numpy.org](https://numpy.org/devdocs/user/whatisnumpy.html)

> NumPy is the fundamental package for scientific computing in Python. 


`numpy` is all about the *ndarray* object which is a homogenous *n*-dimensional array.

`numpy` arrays are less flexible, but much more efficient.

### Vectorization

Vectorized code handles arrays as wholes, instead of handling their elements separately.

Let `a` and `b` be `range` objects.
To multiply all the elements of the first with the corresponding elements of the second we could do one of the following:

In [ ]:
a = range(1000)
b = range(0, 2000, 2)

In [ ]:
%%timeit -n 300

c = []
for i in range(len(a)):
    c.append(a[i]*b[i])

In [ ]:
%%timeit -n 300

c = []
for a_elem, b_elem in zip(a, b):
    c.append(a_elem*b_elem)

In [ ]:
%%timeit -n 300

arr_c = [a_elem*b_elem for a_elem, b_elem in zip(a, b)]

With `numpy` it is possible to write vectorized code, which is much less verbose and even less time consuming.

In [ ]:
import numpy as np

arr_a = np.array(a)
arr_b = np.array(b)

%timeit -n 30000 arr_c = arr_a*arr_b

### The `ndarray`

As mentioned, an ndarray can be $n$-dimensional, so we need introduce some vocabulary to talk about them.

In [ ]:
arr = np.array([[1, 2, 3], [4, 5, 6]])
arr

In [ ]:
arr.shape

In [ ]:
arr.ndim

In [ ]:
arr.size

### Creating arrays

We can create arrays from iterables, or from scratch.

In [ ]:
for outer in (list, tuple):
    for inner in (list, tuple):
        iterable = outer((inner(range(i, i+3)) for i in (1, 4)))
        arr = np.array(iterable)
        print(iterable)
        print(arr)
        print()

In [ ]:
print(np.ones(3), '\n')
print(np.zeros(4), '\n')
print(np.full(5, -1))

We can also specify the data type at creation

In [ ]:
for dtype in (int, float, str, complex, bool):
    arr = np.array((-1, 0, 1), dtype=dtype)
    print(arr.dtype.name.ljust(10), arr, '\n')

### Other things `numpy`:
Let's quickly run through some `numpy` basics

In [ ]:
# Sequences
print(np.arange(1, 10, 2.25))
print(np.linspace(1, 10, 5))
print(np.logspace(0, 1, 5))

In [ ]:
# Operations
A = np.array([[1, 2], [0, 1]])
B = np.array([[4, 1], [0, 4]])
print(A, '\n')
print(B, '\n')
print(A * B, '\n')  # Elementwise
print(A @ B, '\n')  # Matrix operation

In [ ]:
# ndarray methods
arr = np.arange(9).reshape(3, 3)
print(arr)
print(f'sum = {arr.sum()}')
print(f'min = {arr.min()}')
print(f'max = {arr.max()}')

In [ ]:
# Along an axis
print(arr)
for i, func in enumerate(('sum', 'min', 'max', 'mean', 'std')):
    i %= 2
    print(f'{func} along axis {i}: {getattr(arr, func)(axis=i)}')

NB!
By default `np.std()` returns the population standard deviation.
The sample standard deviation (i.e. with  [Bessel's correction](https://en.wikipedia.org/wiki/Bessel%27s_correction)) can be obtained by specifying `ddof=1`:

In [ ]:
print(arr)
print(np.std(arr, axis=1))
print(arr.std(axis=1, ddof=1))

In [ ]:
# Maths
print(np.exp(2))
print(np.sqrt(9))
print(np.log(100))
print(np.log10(100))
print(np.add(2, 2))

### Indexing, slicing, and iterating
1-D arrays are indexed exactly like Python lists.

Multidimensional arrays have one index per axis and are accessed via tuple

In [ ]:
arr = np.arange(9).reshape(3, 3)
print(arr, '\n')

print(arr[2, 2])
print(arr[2, :])
print(arr[2])
print(arr[:, 2])
print(arr[..., 2])

<img src="https://lund-observatory-teaching.github.io/lundpython/imgs/indexing.png" width="1600" style="margin: 0pt 0pt 0pt 20pt;"/>

### Masking

Masking allows us to access elements of an `ndarray` based on some condition.

Suppose we have a one-dimensional sequence of integers and we wish to double all the odd values.

In [ ]:
n_elems = 10

list_ = list(range(n_elems))         # Using list_ avoids overwriting the built-in list
list_ = [elem*2 if elem%2 else elem for elem in list_]

arr = np.arange(n_elems, dtype=int)
arr[arr%2 == 1] *= 2

for seq in (list_, arr):
    print(seq[:10])

But what if the array has more dimensions?

In [ ]:
list_2d = n_elems*[list(range(n_elems))]
list_2d = [[elem*2 if elem%2 else elem for elem in list_] for list_ in list_2d]
print(list_2d[0][:10])
print(list_2d[1][:10])

arr_2d = np.ones((n_elems, 1))*np.arange(n_elems, dtype=int)
arr_2d[arr_2d%2 == 1] *= 2
print(arr_2d[:2, :10])

`numpy` code can often be written in such a way that the number of dimensions an array has is irrelevant.

In [ ]:
def modify_list_recursive(list_):
    if isinstance(list_[0], int):
        return [elem + 2*(elem%2) for elem in list_]
    else:
        return [modify_list_recursive(elem) for elem in list_]

### Stacking

In [ ]:
# 1D arrays
arr1 = np.ones(2)
arr2 = np.zeros(2)
print(np.row_stack((arr1, arr2)), '\n')
print(np.column_stack((arr1, arr2)))

In [ ]:
# 2D arrays
arr1 = np.ones((2, 2))
arr2 = np.zeros((2, 2))
print(np.vstack((arr1, arr2)), '\n')
print(np.hstack((arr1, arr2)), '\n')
print(np.concatenate((arr1, arr2), axis=0), '\n')
print(np.concatenate((arr1, arr2), axis=1))

Only stack arrays if you do not know how large your final array should be.

If you do know the final size then it is much faster to initialize an array with the right shape and fill it with values afterwards.

In [ ]:
def fill_an_array(n, columns):
    arr = np.empty((n, len(columns)), dtype=int)
    # A column can also be given as an array
    for i, col in enumerate(columns):
        arr[:, i] = col
    return arr


def stack_arrays(n, row):
    arr = row.copy()
    for _ in range(n-1):   # Calling the loop variable _ signifies that its value isn't used
        arr = np.vstack((arr, row))
    return arr

In [ ]:
timing = {}
n_rows = (10, 30, 100, 300, 1000)
single_row = np.arange(5)

for func in (fill_an_array, stack_arrays):
    timing[func] = []
    print(func(3, single_row))
    for n in n_rows:
        temp = %timeit -o -q -n 150 func(n, single_row)
        timing[func].append(temp)
    print()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 6))
plt.rcParams.update({'font.size': 22})
for func, results in timing.items():  
    plt.loglog(n_rows, [result.average for result in results],
                 label=func.__name__, marker='o', linestyle='--')
plt.xlabel('Number of rows')
plt.ylabel(r'Average time to run [s]')
plt.legend()
plt.show()

### Shape manipulation

It is not possible to combine arbitrarily shaped `numpy` arrays, so the following might be handy.

In [ ]:
arr = np.arange(12)
print(arr, '\n')

# Reshape
arr_2d = arr.reshape(3, 4)
print(arr_2d, '\n')

# Transpose
arr_T = arr_2d.transpose()
print(arr_T, '\n')

# Flatten
arr_flat = arr_T.flatten()
print(arr_flat, '\n')

### Broadcasting
Broadcasting is how `numpy` treats arrays with different shapes when performing arithmetic operations.
Typically it means that the smaller of the arrays is "stretched" out to match the larger's shape.
For example:
<br>
![](https://lund-observatory-teaching.github.io/lundpython/imgs/np_multiply_broadcasting.png)

This can be rather useful for certain operations. For example consider the illustration below where we are simply doing the operation `arr_c = arr_a * arr_b`  
<br>
![](https://lund-observatory-teaching.github.io/lundpython/imgs/broadcasting.png)

The dimensions of two arrays are compatible with broadcasting when:
<br>
- They are equal
- One of them is 1

So let's see what works

```python
A      (3d array):  15, 3, 5
B      (3d array):  15, 1, 5
Result (3d array):  15, 3, 5
```

```python
A      (3d array):  15, 3, 5
B      (2d array):      3, 5
Result (3d array):  15, 3, 5
```

```python
A      (3d array):  15, 3, 5
B      (2d array):      3, 1
Result (3d array):  15, 3, 5
    
```

```python
A      (2d array):     2, 1
B      (3d array):  8, 4, 3  # second from last dimensions mismatch
```

### Adding dimensions

Broadcasting arrays might require adding new dimensions to them.

Adding a new axis to an array with `np.newaxis` is cheap because it does not change the size of the array.

In [ ]:
arr = np.ones(6)
print(arr.shape, '\n')

print(arr[:, np.newaxis].shape)
print(arr[np.newaxis, :].shape, '\n')

print(np.expand_dims(arr, axis=1).shape)
print(np.expand_dims(arr, axis=0).shape)

### Random numbers
Using the [`numpy.random`](https://numpy.org/devdocs/reference/random/index.html#numpyrandom) module it is possible to generate random numbers from a large number of different distributions.

In [ ]:
from numpy.random import default_rng


rng = default_rng()

# Random numbers between 0 and 1
print(rng.random(3), '\n')

# Random integers
print(rng.integers(low=0, high=10, size=3), '\n')

# Random numbers sampled from uniform distribution
print(rng.uniform(low=0, high=10, size=3), '\n')

# Random numbers sampled from Gaussian/Normal distribution
print(rng.normal(size=3), '\n')

### [SciPy](https://www.scipy.org)

A collection of mathematocal algorithms and convenience functions built on NumPy.  

![](https://lund-observatory-teaching.github.io/lundpython/imgs/scipy_subpackages.png)

### [Astropy](https://www.astropy.org)

Python packages developed for astronomers.  

![](https://lund-observatory-teaching.github.io/lundpython/imgs/astropy.png)

# Go to www.menti.com and enter code: xxxx xxxx

### Exit question 1: Why is NumPy faster than normal Python?

$\quad$<b>A)</b> The code typically has less lines.<br>
$\quad$<b>B)</b> It does things in a non-Python background and never uses any Python commands.<br>
$\quad$<b>C)</b> Because the NumPy ndarray has one data type and is memory-dense.<br>
$\quad$<b>D)</b> It rewrote the existing Python functions in a smarter way.

Correct answer: B & C

### Exit question 2: Which of the options will give you the maximum value of a NumPy array?

$\quad$<b>A)</b> `np.max(my_array)`<br>
$\quad$<b>B)</b> `my_array(np.max)`<br>
$\quad$<b>C)</b> `my_array.max()`<br>
$\quad$<b>D)</b> None of the above.

Correct answer: A & C

### Exit question 3: Arrays `A `and `B` have shapes (8, 1, 6, 1) and (7, 1, 5) respectively. What is the shape of `A*B`?

$\quad$<b>A)</b> (56, 1, 6, 5)<br>
$\quad$<b>B)</b> (8, 7, 6, 5)<br>
$\quad$<b>C)</b> (8, 1, 6, 1, 7, 1, 5)<br>
$\quad$<b>D)</b> `A*B` is not a valid operation.

Correct answer: B

# Now it's time to use the manual to solve the exercises. Good luck!